In [5]:
import os
import sqlite3
import getpass
from dotenv import load_dotenv
from pyprojroot import here

# 导入 LangChain 相关模块
from langchain.chat_models import init_chat_model  # 初始化 LLM 模型
from langchain_community.utilities.sql_database import SQLDatabase  # 用于加载数据库
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit  # 封装 SQL 工具
from langchain_core.tools import tool  # 装饰器，将函数转换为 LLM 可调用的工具
from langchain.agents import AgentType, create_sql_agent  # 用于创建 SQL 代理

# 1. 加载环境变量，确保 API Key 等配置可用
load_dotenv(override=True)

# 2. 定位数据库文件，并利用 SQLDatabase.from_uri 创建数据库连接对象
db_path = here("data/travel2.sqlite")
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

# 打印数据库方言和可用的表名，便于确认数据库结构
print("数据库方言:", db.dialect)
print("可用的表名:", db.get_usable_table_names())

# 3. 初始化 LLM 模型
if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")
llm = init_chat_model("llama3-70b-8192", model_provider="groq")

# 4. 初始化 SQLDatabaseToolkit，将数据库对象和 LLM 模型绑定
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# 获取数据库的 schema 信息（这里只简单展示表名，实际项目中可扩展更多细节）
schema_info = db.get_usable_table_names()

# 5. 定义预定义工具，限定 LLM 只能调用这些工具查询数据库

@tool
def fetch_user_flight_information(passenger_id: str) -> list[dict]:
    """
    根据乘客ID获取该用户的所有票据信息，包括航班和座位信息。
    
    参数：
      - passenger_id: 字符串类型，如 "8149 604011"
    
    返回：
      - 一个字典列表，每个字典包含票号、航班信息和座位信息等字段。
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    query = """
    SELECT 
        t.ticket_no, t.book_ref,
        f.flight_id, f.flight_no, f.departure_airport, f.arrival_airport, 
        f.scheduled_departure, f.scheduled_arrival,
        bp.seat_no, tf.fare_conditions
    FROM 
        tickets t
        JOIN ticket_flights tf ON t.ticket_no = tf.ticket_no
        JOIN flights f ON tf.flight_id = f.flight_id
        JOIN boarding_passes bp ON bp.ticket_no = t.ticket_no AND bp.flight_id = f.flight_id
    WHERE 
        t.passenger_id = ?
    """
    cursor.execute(query, (passenger_id,))
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    result = [dict(zip(columns, row)) for row in rows]
    cursor.close()
    conn.close()
    return result

@tool
def search_flights(
    departure_airport: str = None,
    arrival_airport: str = None,
    start_time: str = None,  # 格式要求：'YYYY-MM-DD'
    end_time: str = None,
    limit: int = 20,
) -> list[dict]:
    """
    根据条件搜索航班信息。
    
    参数：
      - departure_airport: 出发机场代码（字符串），例如 "JFK"
      - arrival_airport: 到达机场代码（字符串），例如 "BJS" 或 "Beijing"
      - start_time: 起飞时间下限，格式 'YYYY-MM-DD'（例如 "2025-05-02"）
      - end_time: 起飞时间上限，格式 'YYYY-MM-DD'
      - limit: 返回记录的最大数量（整数，默认20）
    
    返回：
      - 一个字典列表，每个字典包含 flights 表的所有字段信息。
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    query = "SELECT * FROM flights WHERE 1 = 1"
    params = []
    if departure_airport:
        query += " AND departure_airport = ?"
        params.append(departure_airport)
    if arrival_airport:
        query += " AND arrival_airport = ?"
        params.append(arrival_airport)
    if start_time:
        query += " AND scheduled_departure >= ?"
        params.append(start_time)
    if end_time:
        query += " AND scheduled_departure <= ?"
        params.append(end_time)
    query += " LIMIT ?"
    params.append(limit)
    cursor.execute(query, params)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    result = [dict(zip(columns, row)) for row in rows]
    cursor.close()
    conn.close()
    return result

# 6. 定义代理提示模板和格式说明
# 这里移除了 {tool_names} 占位符，避免 create_sql_agent 在调用 .format() 时因缺失 tool_names 而报错
prefix_template = f"""你是一个SQL查询代理，负责根据用户提问调用预定义工具进行数据库查询。
你已经加载以下数据库schema信息：
表名及其字段：{schema_info}

目前可用的工具有：
1. fetch_user_flight_information(passenger_id: str)
   - 用于获取用户票据信息，参数 passenger_id 为字符串，例如 "8149 604011"。
2. search_flights(departure_airport: str, arrival_airport: str, start_time: str, end_time: str, limit: int)
   - 用于搜索航班信息，参数：
       * departure_airport: 出发机场代码（字符串）
       * arrival_airport: 到达机场代码（字符串）
       * start_time: 起飞时间下限，格式 'YYYY-MM-DD'
       * end_time: 起飞时间上限，格式 'YYYY-MM-DD'
       * limit: 返回记录数（整数）

请根据用户的问题判断需要调用哪个工具，并确保传入参数名称、数据类型和格式正确。不要自由生成SQL查询语句，只能调用以上工具。
"""

format_instructions_template = """使用以下格式：
Question: 用户的问题
Thought: 分析问题后判断需要调用哪个工具
Action: 调用的工具名称（fetch_user_flight_information 或 search_flights）
Action Input: 传入工具的参数（必须严格符合工具参数要求）
Observation: 工具返回的结果
...
Thought: 得出最终答案
Final Answer: 给出最终答案
"""

# 7. 创建 SQL 代理，将 SQLDatabaseToolkit 与预定义工具整合
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    prefix=prefix_template,
    format_instructions=format_instructions_template,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    extra_tools=[fetch_user_flight_information, search_flights]
)

# 8. 示例调用：用户提问查询航班信息（例如查询到北京且起飞时间晚于2025-05-02的航班）
example_query = "请查询到北京的航班，要求起飞时间晚于2025-05-02"
result = agent.run(example_query)
print("最终结果:", result)


数据库方言: sqlite
可用的表名: ['aircrafts_data', 'airports_data', 'boarding_passes', 'bookings', 'car_rentals', 'flights', 'hotels', 'seats', 'ticket_flights', 'tickets', 'trip_recommendations']


ValueError: Prompt missing required variables: {'tool_names'}